In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip /content/drive/MyDrive/capstone_models/final_data.zip -d /content/drive/MyDrive/capstone_models/final_data

In [ ]:
import numpy as np
import cv2
import os

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
def load_images(directory_path, resize_shape=(128, 128)):
    image_list = []
    files = sorted(os.listdir(directory_path))

    for file in files:
        img_path = os.path.join(directory_path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, resize_shape)
        image_list.append(img)

    return np.array(image_list)

# Loading original images and masks
root = '/content/drive/MyDrive/capstone_models/final_data'
image_folder = root + '/images'
mask_folder = root + '/mask'

images = load_images(image_folder)
masks = load_images(mask_folder)

In [ ]:
images[0].shape

(128, 128, 3)

In [ ]:
masks[0].shape

(128, 128, 3)

In [ ]:
def convert_masks_to_binary(masks):
    binary_masks = []
    for mask in masks:
        gray_mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
        _, binary_mask = cv2.threshold(gray_mask, 127, 255, cv2.THRESH_BINARY)
        binary_masks.append(binary_mask / 255.0)
    return np.array(binary_masks)

masks = convert_masks_to_binary(masks)

In [ ]:
class CottonDataset(Dataset):
  def __init__(self, images, masks, transform=None):
    self.images = images
    self.masks = masks
    self.transform = transform

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    image = self.images[idx]
    mask = self.masks[idx]

    image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
    mask = torch.tensor(mask, dtype=torch.float32)

    if self.transform:
      image = self.transform(image)

    return image, mask

In [ ]:
transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset = CottonDataset(images, masks, transform=transform)

In [ ]:
train_size = int(0.7 * len(dataset))  # 70% for training
val_size = int(0.15 * len(dataset))   # 15% for validation
test_size = len(dataset) - train_size - val_size  # Remaining 15% for testing

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class DoubleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class DownConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConvBlock(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class UpConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=3, stride=2, output_padding=1, padding=1)
        self.conv = DoubleConvBlock(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        _, _, current_height, current_width = x2.shape
        _, _, target_height, target_width = x1.shape
        crop_height = (current_height - target_height) // 2
        crop_width = (current_width - target_width) // 2
        cropped_output = x2[:, :,crop_height:crop_height + target_height, crop_width:crop_width + target_width]

        x = torch.cat([cropped_output, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(OutConv, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

  def forward(self, x):
    return self.conv(x)

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.enc1 = (DoubleConvBlock(in_channels, 64))
        self.enc2 = (DownConvBlock(64, 128))
        self.enc3 = (DownConvBlock(128, 256))
        self.enc4 = (DownConvBlock(256, 512))
        self.bottleneck = (DownConvBlock(512, 1024))
        self.dec4 = (UpConvBlock(1024, 512))
        self.dec3 = (UpConvBlock(512, 256))
        self.dec2 = (UpConvBlock(256, 128))
        self.dec1 = (UpConvBlock(128, 64))
        self.outc = (OutConv(64, out_channels))

    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(x1)
        x3 = self.enc3(x2)
        x4 = self.enc4(x3)
        x5 = self.bottleneck(x4)
        x = self.dec4(x5,x4)
        x = self.dec3(x,x3)
        x = self.dec2(x,x2)
        x = self.dec1(x,x1)
        out = self.outc(x)
        return out

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-7):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, y_pred, y_true):
        y_pred = y_pred.view(-1)
        y_true = y_true.view(-1)

        intersection = (y_pred * y_true).sum()
        dice_coeff = (2. * intersection + self.smooth) / (y_pred.sum() + y_true.sum() + self.smooth)
        dice_loss = 1 - dice_coeff

        return dice_loss

In [ ]:
# unet_model = UNet(in_channels=3, out_channels=1).to(device)
# images, masks = next(iter(dataloader))
# output = unet_model(images.to(device))
# print("Output shape from U-Net:", output.shape)

In [ ]:
model = UNet(in_channels=3, out_channels=1).to(device)
criterion = DiceLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
  model.train()
  train_running_loss = 0

  for images, masks in tqdm(train_loader):
    images = images.to(device)
    masks = masks.to(device)

    optimizer.zero_grad()
    output = model(images)

    loss = criterion(output, masks)

    loss.backward()
    optimizer.step()

    train_running_loss += loss.item()

  epoch_loss = train_running_loss / len(train_loader)

  model.eval()
  val_loss = 0
  with torch.no_grad():
      for val_images, val_masks in val_loader:
          val_images = val_images.to(device)
          val_masks = val_masks.to(device)

          val_output = model(val_images)
          val_loss += criterion(val_output, val_masks).item()

  val_loss = val_loss / len(val_loader)
  print(f"Epoch [{epoch + 1}/{num_epochs}],  Train Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}")

100%|██████████| 41/41 [41:53<00:00, 61.30s/it]


Epoch [1/1],  Train Loss: 0.9158, Validation Loss: 0.8810


In [ ]:
model.eval()
test_predictions = []
test_ground_truths = []
with torch.no_grad():
  for test_images, test_masks in test_loader:
    test_images = test_images.to(device)

    test_pred = model(test_images)

    test_predictions.append(test_pred.cpu().numpy())
    test_ground_truths.append(test_masks.cpu().numpy())

In [ ]:
test_predictions

NameError: name 'test_predictions' is not defined